# 0. Test parameters

In [8]:
import numpy as np

n_cameras = 3
ring_R = 0.1
# put camera on a ring on xy plane around z axis of the rig
theta = np.linspace(0, 360.0, n_cameras, endpoint=False)
camera_axes = 'ros'
rot_ros = np.array([-90.0, 0.0, -90.0])
rot = np.tile(rot_ros, (n_cameras, 1))
rot[:,-1] += theta
trans = np.zeros((n_cameras, 3))
trans[:, 0] = ring_R * np.cos(np.radians(theta))
trans[:, 1] = ring_R * np.sin(np.radians(theta))
camera_translate = trans
camera_rotate = rot

dimenvue_config = {
    # initial pose
    'chassis': {
        # (3,) array
        'world_chassis_translate': [0, 0, 0],
        # (3,) x->y->z order euler angles, extrinsic rotation (R = Rz * Ry * Rx)
        'world_chassis_rotate': [0, 0, 0],
    },
    
    'imu': {
        'chassis_imu_translate' : [0, 0, 0.0],
        'chassis_imu_rotate' : [0, 0, 0],
        'imu_rate' : 400,
        'enable': False,
    },

    'lidar' : {
        'chassis_lidar_translate' : [0, 0, 0.1],
        'chassis_lidar_rotate' : [0, 0, 0],
        'lidar_config_file' : "OS1_REV6_32ch10hz512res_noiseless",
        'enable': True,
    },

    'cameras' : {
        'chassis_cameras_translate' : [0, 0, 0.0],
        'chassis_cameras_rotate' : [0, 0, 0],
        'n_cameras' : n_cameras,
        'camera_translate' : camera_translate,
        'camera_rotate' : camera_rotate,
        'camera_axes' : camera_axes,
        'camera_fps' : 30,
        'calibrate_config': {
            'camera_resolution' : [1920, 1200],
            'camera_intrinsics' : [[957.8, 0.0, 960], [0.0, 957.8, 600], [0.0, 0.0, 1.0]],
            'pixel_size' : 3 * 1e-3,
            'f_stop' : 1.8,
            'focus_distance' : 8.9
        },
        'enable': True,
    }
}
# scene configuration
wall_size = 10.0
cell_size = 2.0
wall_distance = 10

# data serialize and deserialize
PICKLE_FILE = 'lidar_rgb_data.pkl'


# 1. Start Isaac Sim

In [2]:
import sys
import os
import signal

# Get the absolute path of the notebook's directory
notebook_dir = os.path.dirname(os.path.abspath('__file__'))+'/dimenvue_ws/'
# Add the notebook's directory to sys.path
sys.path.append(notebook_dir)
# cd to the notebook's directory
os.chdir(notebook_dir)

from isaacsim import SimulationApp

CONFIG = {
    'headless': False,
    'display_options': 3286
}

# Example for creating a RTX lidar sensor and publishing PointCloud2 data
simulation_app = SimulationApp(CONFIG)
simulation_app.update()

spinning = True
def sigint_handler(sig, frame):
    global spinning
    spinning = False
signal.signal(signal.SIGINT, sigint_handler)

kernel = get_ipython().kernel
kernel.pre_handler_hook = lambda: None
kernel.post_handler_hook = lambda: None

def spin(update_function, *args, **kwargs):
    global spinning
    print('gui can be used now. interrupt kernel to go back to notebook')
    while spinning:
        update_function(*args, **kwargs)
    spinning = True

def resume_gui():
    spin(lambda: simulation_app.update())
    
if CONFIG['headless']:
    del spin
    del resume_gui

Starting kit application with the following args:  ['/f/ov/pkg/isaac-sim-4.1.0/exts/omni.isaac.kit/omni/isaac/kit/simulation_app.py', '/f/ov/pkg/isaac-sim-4.1.0/apps/omni.isaac.sim.python.kit', '--/app/tokens/exe-path=/f/ov/pkg/isaac-sim-4.1.0/kit', '--/persistent/app/viewport/displayOptions=3286', '--/rtx/materialDb/syncLoads=True', '--/rtx/hydra/materialSyncLoads=True', '--/omni.kit.plugin/syncUsdLoads=True', '--/app/renderer/resolution/width=1280', '--/app/renderer/resolution/height=720', '--/app/window/width=1440', '--/app/window/height=900', '--/renderer/multiGpu/enabled=True', '--/app/fastShutdown=False', '--ext-folder', '/f/ov/pkg/isaac-sim-4.1.0/exts', '--ext-folder', '/f/ov/pkg/isaac-sim-4.1.0/apps', '--/physics/cudaDevice=0', '--portable']
Passing the following args to the base kit application:  ['-f', '/home/linh/.local/share/jupyter/runtime/kernel-ee864e3c-656a-497b-9aac-8c23b2fe7553.json']
[Warning] [omni.isaac.kit.simulation_app] Non-headless mode not supported with jupyt

Warp 1.2.1 initialized:
   CUDA Toolkit 11.8, Driver 12.5
   Devices:
     "cpu"      : "x86_64"
     "cuda:0"   : "NVIDIA GeForce RTX 4060" (8 GiB, sm_89, mempool enabled)
   Kernel cache:
     /home/linh/.cache/warp/1.2.1
 [ext: omni.kit.widget.text_editor-1.0.2] startup
[3.092s] [ext: omni.kit.viewport.window-106.0.8] startup
[3.115s] [ext: omni.kit.window.property-1.11.1] startup
[3.117s] [ext: omni.usdphysics-106.0.20] startup
[3.122s] [ext: omni.graph.tools-1.78.0] startup
[3.148s] [ext: omni.kit.viewport.utility-1.0.17] startup
[3.148s] [ext: omni.kit.property.usd-3.21.28] startup
[3.159s] [ext: omni.graph-1.135.0] startup
[3.213s] [ext: omni.graph.image.core-0.3.2] startup
[3.219s] [ext: omni.graph.ui-1.70.0] startup
[3.236s] [ext: omni.graph.action_core-1.1.4] startup
[3.247s] [ext: omni.kit.window.cursor-1.1.2] startup
[3.249s] [ext: omni.graph.image.nodes-1.0.2] startup
[3.252s] [ext: omni.graph.action_nodes-1.23.0] startup
[3.263s] [ext: omni.kit.viewport.menubar.core-106.0

In [3]:
import os
import glob
import json

import carb
import omni
from pxr import Gf, Sdf
import omni.graph.core as og
import omni.replicator.core as rep
from omni.isaac.core import SimulationContext
from omni.isaac.core.objects import *
from omni.isaac.sensor import *
from omni.isaac.core.utils.stage import *
from omni.isaac.core.utils.prims import *
from omni.isaac.core.utils.viewports import *
from omni.isaac.core.utils.xforms import get_world_pose
from omni.isaac.nucleus import get_assets_root_path
import numpy as np

from rotations import *
from dimenvue import make_dimenvue


# 3. Capture LiDAR-RGB sample
## 3.1. Setup simulation
The simulation stage set up as follow:
- Build the dimenvue device
- Put some object to for camera to see
- Put some light to make object visible

In [62]:
def setup_scene(wall_size, cell_size=1.0, dt=1.0/30):
    create_new_stage()
    
    ctx = SimulationContext(physics_dt=dt, rendering_dt=dt)
    
    h = w = int(wall_size / cell_size)
    # stack a wall of cubes with different colors on YZ plane
    wall = create_prim(
        prim_path = '/wall'
    )
    
    colors = np.random.rand(h, w, 3)
    for i in range(h):
        for j in range(w):
            FixedCuboid(
                f'/wall/cube_{i}_{j}',
                translation=[0, cell_size*i, cell_size*j],
                size=cell_size,
                color=np.array(colors[i, j])
            )

    FixedCone('/cone',
              position = [-12, 12, 0],
              radius = 5.0,
              height=8.0, 
              color=np.array([1, 0, 0]))
    
    FixedCylinder('/cylinder',
                  position = [-12, -12, 0],
                  radius = 4.0,
                  height=10.0,
                  color = np.array([0, 1, 0]))
    
    create_prim(
        prim_path='/environment/light',
        prim_type="SphereLight",
        position=[0, 0, 0],
        attributes={
            "inputs:radius": 0.5,
            "inputs:intensity": 300000
        }
    )
    
    dimenvue = make_dimenvue(dimenvue_config)
    
    return ctx, wall, dimenvue

## 3.3. Define test

In [64]:
import numpy as np
import copy

def capture_lidar_rgb(wall_size, cell_size=1.0, wall_distance=10):
    ctx, wall, dimenvue = setup_scene(wall_size, cell_size)
    wall.GetAttribute('xformOp:translate').Set(Gf.Vec3d(wall_distance, -wall_size/2, -wall_size/2))
    
    lidar = dimenvue['lidar']
    # lidar.enable_visualization()
    cameras = dimenvue['cameras']
    chassis = dimenvue['chassis']
    
    data = {
        'lidar': None,
        'cameras': {}
    }
    
    def log_points(lidar_data):
        data['lidar'] = copy.deepcopy(lidar_data)
        
    lidar.set_frame_ready_callback(log_points)

    ctx.play()
    for _ in range(30):
        ctx.step()
    ctx.stop()
    
    for cam in cameras:
        data['cameras'][str(cam.prim_path).split('/')[-1]] = cam.get_rgba()
    
    return data
  

## 3.4. Run tests

In [65]:
data = capture_lidar_rgb(wall_size, cell_size, wall_distance)

In [13]:
resume_gui()

gui can be used now. interrupt kernel to go back to notebook


# 4. Align Lidar and RGB
## 4.1. Save sim data for later


In [10]:
import pickle

with open(PICKLE_FILE, 'wb') as f:
    pickle.dump(data, f)


## 4.2. Reload data
Run cell in section 0 first to set parameters

In [11]:
# run
import pickle

with open(PICKLE_FILE, 'rb') as f:
    data = pickle.load(f)


## 4.3. Log to rerun for visualization

In [48]:
import rerun as rr
import numpy as np

rr.init('test_lidar_rgb', spawn=True)

[2024-09-06T04:40:34Z INFO  re_sdk_comms::server] Hosting a SDK server over TCP at 0.0.0.0:9876. Connect with the Rerun logging SDK.
[2024-09-06T04:40:34Z INFO  winit::platform_impl::platform::x11::window] Guessed window scale factor: 1
[2024-09-06T04:40:34Z INFO  re_sdk_comms::server] New SDK client connected from: 127.0.0.1:58850


[2024-09-06T04:40:34Z WARN  wgpu_hal::gles::egl] No config found!
[2024-09-06T04:40:34Z WARN  wgpu_hal::gles::egl] EGL says it can present to the window but not natively
[2024-09-06T04:40:34Z INFO  egui_wgpu] There were 3 available wgpu adapters: {backend: Vulkan, device_type: Cpu, name: "llvmpipe (LLVM 12.0.0, 256 bits)", driver: "llvmpipe", driver_info: "Mesa 21.2.6 (LLVM 12.0.0)", vendor: 0x10005}, {backend: Vulkan, device_type: DiscreteGpu, name: "NVIDIA GeForce RTX 4060", driver: "NVIDIA", driver_info: "555.42.02", vendor: 0x10DE, device: 0x2882}, {backend: Gl, device_type: Other, name: "NVIDIA GeForce RTX 4060/PCIe/SSE2", driver: "OpenGL", driver_info: "4.6.0 NVIDIA 555.42.02", vendor: 0x10DE}


In [49]:
"""
Args:
    pos: translation
    ori: [rx, ry, rz] rotation around parent xyz axes in degrees (extrinsic convention)
"""
def rr_transform(pos:np.ndarray, ori:np.ndarray):
    wxyz = euler_angles_to_quat(ori, degrees=True, extrinsic=True)
    xyzw = np.roll(wxyz, -1)
    return rr.Transform3D(
        translation=pos,
        quaternion=xyzw
    )

def Axes(length=1.0):
    return rr.Arrows3D(
        vectors=[[length, 0, 0], [0, length, 0], [0, 0, length]],
        colors=[[255,0,0], [0,255,0], [0,0,255]],
    )


In [50]:
import rotations

def rr_transform(pos:np.ndarray, ori:np.ndarray):
    wxyz = euler_angles_to_quat(ori, degrees=True, extrinsic=True)
    xyzw = np.roll(wxyz, -1)
    return rr.Transform3D(
        translation=pos,
        quaternion=xyzw
    )

def rr_set_up_frame():
    rr.set_time_seconds('sim_time', 0.0)
    rr.log('/world', rr.Clear(recursive=True))
    rr.log('/world', Axes(1), static=True)
    rr.log('/world/chassis', Axes(1))
    if 'lidar' in dimenvue_config:
        c = dimenvue_config['lidar']
        p, r = c['chassis_lidar_translate'], c['chassis_lidar_rotate']
        rr.log('/world/chassis/lidar', rr_transform(p, r))
        rr.log('/world/chassis/lidar', Axes(1))
    if 'cameras' in dimenvue_config:
        c = dimenvue_config['cameras']
        p, r = c['chassis_cameras_translate'], c['chassis_cameras_rotate']
        rr.log('/world/chassis/cameras', rr_transform(p, r))
        for i in range(c['n_cameras']):
            p, r = c['camera_translate'][i], c['camera_rotate'][i]
            rr.log(f'/world/chassis/cameras/cam{i}', rr_transform(p, r))
            rr.log(f'/world/chassis/cameras/cam{i}', Axes(1))
            c1 = c['calibrate_config']
            ((fx, _, cx),(_,fy,cy),(_,_,_)) = c1['camera_intrinsics']
            rr.log(f'/world/chassis/cameras/cam{i}/image', rr.Pinhole(
                resolution=c1['camera_resolution'],
                camera_xyz=rr.ViewCoordinates.RDF, #same as ROS
                focal_length=(fx, fy),
                principal_point=(cx, cy),
                image_plane_distance=wall_distance - cell_size/2 - ring_R
            ))

rr_set_up_frame()

In [52]:
def tf(translate, rotate):
    tf = np.eye(4)
    tf[:3, 3] = translate
    tf[:3, :3] = euler_to_rot_matrix(rotate, degrees=True, extrinsic=True)
    return tf

def get_transforms(config):
    c = config['lidar']
    p, r = c['chassis_lidar_translate'], c['chassis_lidar_rotate']
    tf_chassis_lidar = tf(p, r)
    c = config['cameras']
    p, r = c['chassis_cameras_translate'], c['chassis_cameras_rotate']
    tf_chassis_cameras = tf(p, r)
    tf_camera_lidar = []
    for i in range(c['n_cameras']):
        p, r = c['camera_translate'][i], c['camera_rotate'][i]
        tf_chassis_camera_i = np.dot(tf_chassis_cameras, tf(p, r))
        tf_camera_i_chassis = np.linalg.inv(tf_chassis_camera_i)
        tf_camera_i_lidar = np.dot(tf_camera_i_chassis, tf_chassis_lidar)
        tf_camera_lidar.append(tf_camera_i_lidar)
    
    return tf_camera_lidar        


tf_camera_lidar = get_transforms(dimenvue_config)

def to_frame(points, tf):
    return np.dot(tf[:3, :3], points.T).T + tf[:3, 3]
    

In [68]:
intrinsic = dimenvue_config['cameras']['calibrate_config']['camera_intrinsics']
points = data['lidar']['point_cloud_data']
rgb = data['cameras']['cam0']
tf = tf_camera_lidar[0]

def colorize(points, rgb, intrinsic, tf):
    ((fx, _, cx),(_,fy,cy),(_,_,_)) = intrinsic
    w, h = cx*2, cy*2
    # move points from lidar frame to camera frame
    points_cam = to_frame(points, tf)
    # remove the points behind the camera
    front_points_idx = np.where(points_cam[:, 2] > 0)[0]
    front_points = points_cam[front_points_idx]
    # x = x/z, y = y/z, z = 1
    points_xy = front_points / front_points[:, 2][:, None]
    # project to image plane
    u = points_xy[:, 0] * fx + cx
    v = points_xy[:, 1] * fy + cy
    u = np.round(u).astype(int)
    v = np.round(v).astype(int)

    inframe_points_idx = np.where((u >= 0) & (u < w) & (v >= 0) & (v < h))[0]
    u = u[inframe_points_idx]
    v = v[inframe_points_idx]
    colors = rgb[v, u]

    points_idx = front_points_idx[inframe_points_idx]
    colored_points = points[points_idx]

    return colored_points, colors, points_idx

color_points, colors, idx = colorize(points, rgb, intrinsic, tf)
rr.log('/world/chassis/lidar/points_color', rr.Points3D(
    positions=color_points, 
    colors=colors)
)


In [69]:
intrinsic = dimenvue_config['cameras']['calibrate_config']['camera_intrinsics']
n_cams = dimenvue_config['cameras']['n_cameras']
points = data['lidar']['point_cloud_data']
rr.log('/world/chassis/lidar/points', rr.Points3D(positions=points))
result_points = []
result_colors = []
for i in range(n_cams):
    rgb = data['cameras'][f'cam{i}']
    rr.log(f'/world/chassis/cameras/cam{i}/image', rr.Image(rgb))
    tf = tf_camera_lidar[i]
    color_points, colors, idx = colorize(points, rgb, intrinsic, tf_camera_lidar[i])
    result_points.append(color_points)
    result_colors.append(colors)
    points = np.delete(points, idx, axis=0)

points = np.concatenate(result_points)
colors = np.concatenate(result_colors)
rr.log(f'/world/chassis/lidar/points_color', rr.Points3D(
    positions=points, 
    colors=colors)
)
    


In [70]:
resume_gui()


gui can be used now. interrupt kernel to go back to notebook


: 

: 